In [1]:
import pandas as pd
import os #기타 운영 체제 인터페이스
import sys #시스템 특정 파라미터와 함수
sys.path.append("../")
sys.path.append("../..") #path는 모듈의 검색 경로를 지정하는 문자열 리스트

from pathlib import Path

import matplotlib.pyplot as plt 
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

# 1. DB 준비

In [2]:
from KETIPreDataIngestion.KETI_setting import influx_setting_KETI as ins
from influxdb_client.client.write_api import SYNCHRONOUS, ASYNCHRONOUS
from influxdb_client import InfluxDBClient, Point, BucketsService, Bucket

version = 2 # or 1

if version == 1: 
    from KETIPreDataIngestion.data_influx import influx_Client as influx_Client
    dataServer = ins.CLUSTDataServer
elif version == 2:
    from KETIPreDataIngestion.data_influx import influx_Client_v2 as influx_Client
    dataServer = ins.CLUSTDataServer2
    
DBClient = influx_Client.influxClient(dataServer)

#DBClient.measurement_list("test_so7")
#DBClient.delete_measurement("test_so3")

# 2. 데이터 경로 탐색

In [3]:
mainDir ='StrawOpenData'
entries = os.listdir(mainDir) 
#listdir는 지정한 디렉토리 내의 모든 파일과 디렉토리의 리스트를 리턴한다.
entries = Path(mainDir)

In [4]:
pathList=[]

#os.walk : 디렉토리, 파일 탐색
#
for path, subdirs, files in os.walk(mainDir):
    for name in files:
        fileName = os.path.join(path, name) #join, 인수에 전달된 2개의 문자열을 결합하여 1개의 경로로 만듦.
        pathList.append(fileName)

In [5]:
"""count1 = 0
for x in pathList
    count1 += 1
    print(count1)
"""

'count1 = 0\nfor x in pathList\n    count1 += 1\n    print(count1)\n'

# 3. 각 데이터를 읽고 저장 

In [6]:
### 1. Database Name : Farm_Strawberry_OpenData

### 2. Measurement Name: DeaejewonYuseongGu_PF_0020699_01
bk_name="farm_straberry_openSo"
DBClient.create_bucket(bk_name)

#컬럼 1번과, 2번 확인 후 한글명을 영문명으로 변경한다.
dict_sido_name = { '경기도' : 'Gyeonggido', '경상남도' : 'Gyeongsangnamdo', '경상북도' : 'Gyeongsangbukdo', 
             '광주광역시' : 'Gwangju', '대전광역시' : 'Daejeon', '전라남도' : 'Jeollanamdo', '전라북도' : 'Jeollabukdo',
             '충청남도' : 'Chungcheongnamdo', '충청북도' : 'Chungcheongbukdo'
    
}
dict_sigungu_name = { '화성시' : 'hwaseong', '연천군' : 'yeoncheon', '광주시' : 'gwangju', 
                '안산시' : 'ansan', '양주시' : 'yangju', '하동군' : 'hadong', '밀양시' : 'miryang', 
                '진주시' : 'jinju', '군위군' : 'gunwi', 
                '청도군' : 'cheongdo', '상주시' : 'sangju', '광산구' : 'gwangsan', '유성구' : 'yuseong', 
                '장성군' : 'jangseong', '보성군' : 'boseong', '화순군' : 'hwasun', '담양군' : 'damyang', 
                '나주시' : 'naju', '강진군' : 'gangjin', '영광군' : 'yeonggwang', '고흥군' : 'goheung', 
                '영암군' : 'yeongam', 
                '김제시' : 'gimje', '부안군' : 'buan', '완주군' : 'wanju',
                '천안시' : 'cheonan', '홍성군' : 'hongseong', '보령시' : 'boryeong', '부여군' : 'buyeo', 
                '아산시' : 'asan', 
                '진천군' : 'jincheon', '옥천군' : 'okcheon', '청주시' : 'cheongju'
    
}

count = 0
for x in pathList :
 
    count += 1
    fileName =  x
    df_original = pd.read_csv(fileName, encoding = 'CP949', parse_dates=['수집일'], index_col='수집일') 
    #df_second = pd.read_csv(fileName, encoding = 'CP949', skiprows=1, nrows=1, usecols = [0,1,2]) 

    df_city = df_original['시도'].unique()[0] 
    df_sigungu = df_original['시군구'].unique()[0]
    df_id = df_original['시설ID'].unique()[0]

    city_name_en = ''
    if  df_city in dict_sido_name :
        city_name_en = dict_sido_name[df_city]    

    sigungu_name_en = ''
    if  df_sigungu in dict_sigungu_name :
        sigungu_name_en = dict_sigungu_name[df_sigungu].capitalize()

    #print(city_name_en, sigungu_name_en)
    #print(df_second.columns[0] + '_' + df_second.columns[1] + '_' + df_second.columns[2])

    # 추후 csv 파일 이름 일괄 변경시 사용하기 ----------------------------------------------------
    #old_file_name = pathList[0]
    #dir_name = os.path.dirname(old_file_name)
    #new_file_name = df_id + '_' + city_name_en + sigungu_name_en + '.csv'
    #new_file_name = dir_name + "/" + new_file_name
    #os.rename(old_file_name, new_file_name) 
    #--------------------------------------------------------------------------------------------
    
    fileName = df_id + '_' + city_name_en + sigungu_name_en
    
    df_original.index.names = ["time"]
    renameColumnList = {'외부온도':"outTemperature", '외부습도':"outHumidity", '외부 일사량':"outSunshine", '외부풍향':"outWindDirection", 
                    '외부풍속':"outWindSpeed", '외부강우':"outRainFall", '강우감지':"outRainDetection", 
                    '내부온도':"inTemperature", '내부습도':"inHumidity", '내부CO2':"inCO2", 
                    '내부 일사량':"inSunshine", '토양EC':"soilEC", '토양PH':"soilPH", 
                    '지온':"groundTemperature", '지습':"groundHumidity", '(양액)총배액':"totalDrainedSolutionEC",                  
                    '(양액)총급액':"totalIrrigatedSolutionEC", '(양액)배액PH':"drainedSolutionPH", 
                    '(양액)공급PH':"IrrigatedSolutionPH", 
                    '(양액)배액EC':"drainedSolutionEC", 
                    '(양액)공급EC':"IrrigatedSolutionEC", 
                    '(양액)지온':"solutionTemperature",
                    '(양액)지습':"solutionHumidity", 
                    '초장':"Chojang", '엽장':"YeopJang", '엽폭':"YeopPok",
                    '엽병장':"YeopBeyungJang", '엽수':"YeopSu", '관부직경':"GwanbuJikKung", 
                    '화방 꽃수(소화수)':"flowerLeafsNum", 
                    '착과수':"fruitsNum", '최종화방차수':"finalHwabangNum"}

    drop_columns = ['시설ID', '시도', '시군구', '품목명', '품종', '재배면적', '시설유형', '재배형태', '단동/연동 구분', '배기팬작동여부', 'CO2발생장치작동여부', '보광등작동여부', '난방기작동상태', '난방기작동여부',
       '유동팬작동상태', '유동팬작동여부', '보온커튼작동상태', '보온커튼작동여부', '차광커튼작동상태', '차광커튼작동여부',
       '우측삼중측창 작동여부', '좌측삼중측창 작동여부', '우측이중측창 작동여부', '좌측이중측창 작동여부', '우측일중측창 개도',
       '우측일중측창 온도폭', '우측일중측창 설정온도', '우측일중측창 작동여부', '우측일중측창 작동상태', '좌측일중측창 개도',
       '좌측일중측창 온도폭', '좌측일중측창 설정온도', '좌측일중측창 작동여부', '좌측일중측창 작동상태', '측창',
       '우측이중천창 작동여부', '좌측이중천창 작동여부', '우측일중천창 개도', '우측일중천창 온도폭', '우측일중천창 설정온도',
       '우측일중천창 작동여부', '우측일중천창 작동상태', '좌측일중천창 개도', '좌측일중천창 온도폭', '좌측일중천창 설정온도',
       '좌측일중천창 작동여부', '좌측일중천창 작동상태', '천창', '조사일', '주차']
    
    df = df_original.rename(columns=renameColumnList)
    df = df.drop(drop_columns, axis=1).dropna(axis=1, how='all')

    #print("count:: ", count, fileName)
    DBClient.write_db_2(bk_name, fileName, df)
 



========== create bucket ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ===

In [ ]:
DBClient.close_db()

In [ ]:

#1,2,3 열의 두 번째 행 읽기
"""
df_original.index.names = ["time"]
renameColumnList = {'외부온도':"outTemperature", '외부습도':"outHumidity", '외부 일사량':"outSunshine", '외부풍향':"outWindDirection", 
                    '외부풍속':"outWindSpeed", '외부강우':"outRainFall", '강우감지':"outRainDetection", 
                    '내부온도':"inTemperature", '내부습도':"inHumidity", '내부CO2':"inCO2", 
                    '내부 일사량':"inSunshine", '토양EC':"soilEC", '토양PH':"soilPH", 
                    '지온':"groundTemperature", '지습':"groundHumidity", '(양액)총배액':"totalDrainedSolutionEC",                  
                    '(양액)총급액':"totalIrrigatedSolutionEC", '(양액)배액PH':"drainedSolutionPH", 
                    '(양액)공급PH':"IrrigatedSolutionPH", 
                    '(양액)배액EC':"drainedSolutionEC", 
                    '(양액)공급EC':"IrrigatedSolutionEC", 
                    '(양액)지온':"solutionTemperature",
                    '(양액)지습':"solutionHumidity", 
                    '초장':"Chojang", '엽장':"YeopJang", '엽폭':"YeopPok",
                    '엽병장':"YeopBeyungJang", '엽수':"YeopSu", '관부직경':"GwanbuJikKung", 
                    '화방 꽃수(소화수)':"flowerLeafsNum", 
                    '착과수':"fruitsNum", '최종화방차수':"finalHwabangNum"}

#df_city = df_original['시도'].unique() + df_original['시군구'].unique()
#df_id = df_original['시설ID'].unique() 

drop_columns = ['시설ID', '시도', '시군구', '품목명', '품종', '재배면적', '시설유형', '재배형태', '단동/연동 구분', '배기팬작동여부', 'CO2발생장치작동여부', '보광등작동여부', '난방기작동상태', '난방기작동여부',
       '유동팬작동상태', '유동팬작동여부', '보온커튼작동상태', '보온커튼작동여부', '차광커튼작동상태', '차광커튼작동여부',
       '우측삼중측창 작동여부', '좌측삼중측창 작동여부', '우측이중측창 작동여부', '좌측이중측창 작동여부', '우측일중측창 개도',
       '우측일중측창 온도폭', '우측일중측창 설정온도', '우측일중측창 작동여부', '우측일중측창 작동상태', '좌측일중측창 개도',
       '좌측일중측창 온도폭', '좌측일중측창 설정온도', '좌측일중측창 작동여부', '좌측일중측창 작동상태', '측창',
       '우측이중천창 작동여부', '좌측이중천창 작동여부', '우측일중천창 개도', '우측일중천창 온도폭', '우측일중천창 설정온도',
       '우측일중천창 작동여부', '우측일중천창 작동상태', '좌측일중천창 개도', '좌측일중천창 온도폭', '좌측일중천창 설정온도',
       '좌측일중천창 작동여부', '좌측일중천창 작동상태', '천창', '조사일', '주차']
df = df_original.rename(columns=renameColumnList)
df = df.drop(drop_columns, axis=1).dropna(axis=1, how='all')

#print(fileName)
DBClient.write_db_2(bk_name, fileName, df)

"""

In [ ]:
### 4. Delete Data DB, MS
